In [25]:
import json
import numpy as np
import pandas as pd

In [12]:
train_df = pd.read_csv("train_10000.txt", header=None)
test_df = pd.read_csv("test_10000.txt", header=None)

train_df.columns = test_df.columns = ["label", "url"]
print(len(train_df), len(test_df))

49850 2624


In [13]:
from urllib.parse import urlparse
import tldextract
import requests
import time
from tqdm import tqdm
url = 'https://www.virustotal.com/vtapi/v2/domain/report'
apikey = ["e871f51212bd7cc4b3580aa7a4668776988b67b7006f97b3b6ba8093a3c7c212", "5ce11f4b6fc5496fa97dc610f448146af2f8bf8397d13f7c7af2fea41651139c"]
apikeyidx = 0

In [14]:
train_whois_json_response = {}
test_whois_json_response = {}

In [15]:
cnt = 0
for i in tqdm(train_df.url.values):
    if i not in train_whois_json_response:
        try:
            while True:
                domain = urlparse(i).netloc
                params = {'apikey': apikey[apikeyidx%2], 'domain':domain}
                response = requests.get(url, params=params)
                if response.status_code == 204:
                    cnt += 1
                    time.sleep(1)
                    if cnt > 100:
                        apikeyidx += 1
                        cnt = 0
                else:
                    break
            train_whois_json_response[i] = response.json()
        except KeyboardInterrupt:
            break

100%|██████████| 49850/49850 [5:22:31<00:00,  2.58it/s]


In [16]:
cnt = 0
for i in tqdm(test_df.url.values):
    if i not in test_whois_json_response:
        try:
            while True:
                domain = urlparse(i).netloc
                params = {'apikey': apikey[apikeyidx%2], 'domain':domain}
                response = requests.get(url, params=params)
                if response.status_code == 204:
                    cnt += 1
                    time.sleep(1)
                    if cnt > 100:
                        apikeyidx += 1
                        cnt = 0
                else:
                    break
            test_whois_json_response[i] = response.json()
        except KeyboardInterrupt:
            break

100%|██████████| 2624/2624 [19:16<00:00,  2.27it/s]


In [18]:
len(train_whois_json_response), len(test_whois_json_response)

(49846, 2624)

In [97]:
minset = Counter()
cnt = 0

for url in train_df.url.values[1:]:
    tmp = train_whois_json_response[url]
    if tmp["response_code"] == 0 or tmp['whois'] == None:
        continue
    tmp = tmp['whois'].split('\n')
    tmp = [i.split(':')[0].lower() for i in tmp]
    minset += Counter(set(tmp))

print(minset.most_common(30))

[('domain name', 38692), ('dnssec', 38208), ('registrar', 37332), ('name server', 36466), ('domain status', 35776), ('creation date', 35225), ('updated date', 35147), ('registrar url', 34840), ('registry domain id', 34743), ('registrar iana id', 34429), ('registrar whois server', 34116), ('registry expiry date', 33753), ('registrar abuse contact email', 33617), ('registrar abuse contact phone', 33598), ('registrant country', 30592), ('registrar registration expiration date', 29620), ('registrant state/province', 26958), ('registrant organization', 26521), ('registrant email', 25298), ('tech country', 23260), ('admin country', 23218), ('tech organization', 22168), ('tech state/province', 22155), ('admin state/province', 22081), ('admin organization', 22031), ('admin city', 21083), ('tech city', 21072), ('registrant city', 20011), ('registrant name', 19930), ('registrant phone', 19501)]


In [102]:
train_whois_json_response[train_df.url.values[1]]['whois']

'DNSSEC: unsigned\nDomain Name: mtsx.com.cn\nDomain Status: ok\nExpiration Time: 2021-08-20 17:36:12\nName Server: ns1.myhostadmin.net\nName Server: ns2.myhostadmin.net\nRegistrant Contact Email: 80d531aef57a4037\nRegistrant ID: 4cd3e40d1b828e90\nRegistrant: 0fa8c9414773127c\nRegistration Time: 2012-08-20 17:36:12\nSponsoring Registrar: 北京新网数码信息技术有限公司'

In [99]:
minset['status']

4690

In [110]:
d = set(["domain name", "domain"])
n = set(["name server", "nserver"])
s = set(['domain status', 'status'])
w = set(["dnssec", "registrar", "registrar whois server", 'registrant country'])

In [114]:
whois = {}

for url in train_df.url.values[1:]:
    response = train_whois_json_response[url]
    if response["response_code"] == 0 or response['whois'] == None:
        continue
    tmp = response['whois'].split('\n')
    keys = set([i.split(':')[0].lower() for i in tmp])
    if len(w - keys) == 0:
        try:
            whois[url] = {}
            for i in tmp:
                key = i.split(':')[0].strip().lower()
                value = i.split(':')[1].strip().lower()
                if key in d:
                    key = "domain name"
                    whois[url][key] = value
                elif key in s:
                    key = "domain status"
                    whois[url][key] = value
                elif key in s:
                    key = "name server"
                    whois[url][key] = value
                elif key in w:
                    whois[url][key] = value
        except:
            pass
            

print(len(whois))

29686
